In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sat Sep 21 07:39:17 2018


@author: Zhenkai Wang(kay)
this code is redeveloped from the code in following links:
https://blog.csdn.net/Eastmount/article/details/50891162
https://blog.csdn.net/Eastmount/article/details/50824215
try to use a lda model in word-freq-matrix
"""

import os  
import sys
import numpy as np
import matplotlib
import scipy
import matplotlib.pyplot as plt
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer 
import pickle, json


if __name__ == "__main__":
 
#    filePath = "listOfWordLists.dat"
#    fileNames = pickle.load(open(fileNamesPath, "rb"))  
    wordListsPath = "Abbreviation_from_files.dat"
    fileNamesPath = "computedFiles.dat"
    listOfWordListsDict = pickle.load(open(wordListsPath, "rb"))
    listOfWordLists = []
    fileNames = []
    
    for file, wordList in listOfWordListsDict.items():
#        print(wordList)
        listOfWordLists.append(wordList)
        fileNames.append(file)
    print("num of docs:", len(listOfWordLists))
    minCount = 5;
    vectorizer = CountVectorizer(min_df = minCount,tokenizer=lambda doc: doc, lowercase=False)
    X= vectorizer.fit_transform(listOfWordLists)
    print(X.shape)

#    X = vectorizer.fit_transform(corpus)
    analyze = vectorizer.build_analyzer()
    weight = X.toarray()
 
    print(len(weight))
#    print(weight[:5, :5])
 
    #LDA算法
    print('LDA:')
    import lda
    import lda.datasets
    numTopic = 5
    model = lda.LDA(n_topics= numTopic, n_iter=500, random_state=1)
    model.fit(np.asarray(weight))     # model.fit_transform(X) is also available
    topic_word = model.topic_word_    # model.components_ also works
 
    #文档-主题（Document-Topic）分布
    doc_topic = model.doc_topic_
#    print("type(doc_topic): {}".format(type(doc_topic)))
    print("shape: {}".format(doc_topic.shape))
 
#    print("file num", len(fileNames))

 
    #输出前10篇文章最可能的Topic
    labels = []      
    for n in range(len(listOfWordLists)):
        topic_most_pr = doc_topic[n].argmax()
        labels.append(topic_most_pr)
#        print("doc: {},topic: {}".format(n, topic_most_pr))
    
    #count label distribution
    labelMap = {}
    for item in labels:
        labelMap[item] = labelMap.get(item, 0) + 1
    print("stats of topics amony documents:")
    print(labelMap)
    # one result: {1: 176, 4: 104, 2: 55, 0: 93, 3: 70}
    #write the label to file
    with open("labels.txt", "w" ) as f:
        for i in range(len(labels)):
            item = labels[i]
            file = fileNames[i]
            f.write("%s\t:%s\n" %(file, item))
    #write down a dictionary, whose key is the topic index, value is the list of files 
    #classified into this topic
    with open("labels2.txt", "w") as f:
        tempDict = {}
        for i in range(numTopic):
            tempDict[i] = []
        for i in range(len(labels)):
            item = labels[i]
            file = fileNames[i]
            tempDict[item].append(file)
        json.dump(tempDict, f)
    #match the label to document name
    
    
        
    #计算文档主题分布图
    import matplotlib.pyplot as plt  
#    f, ax= plt.subplots(6, 1, figsize=(8, 8), sharex=True)  
#    for i, k in enumerate([0, 1, 2, 3, 8, 9]):  
#        ax[i].stem(doc_topic[k,:], linefmt='r-',  
#                   markerfmt='ro', basefmt='w-')  
#        ax[i].set_xlim(-1, 2)     #x坐标下标
#        ax[i].set_ylim(0, 1.2)    #y坐标下标
#        ax[i].set_ylabel("Prob")  
#        ax[i].set_title("Document {}".format(k))  
#    ax[5].set_xlabel("Topic")
#    plt.tight_layout()
#    plt.show() 
    
    #show the top words in each topic    
#    featureName = pickle.load(open("featureName.dat", "rb"))
    vocabulary = vectorizer.vocabulary_;
    tempDict = {};
    for word, idx in vocabulary.items():
        tempDict[idx] = word
    featureNames = []
    for i in range(len(tempDict)):
        featureNames.append(tempDict[i])
#    print(featureNames)






num of docs: 400
(400, 347)
400
LDA:


INFO:lda:n_documents: 400
INFO:lda:vocab_size: 347
INFO:lda:n_words: 83226
INFO:lda:n_topics: 5
INFO:lda:n_iter: 500
INFO:lda:<0> log likelihood: -514341
INFO:lda:<10> log likelihood: -360264
INFO:lda:<20> log likelihood: -347698
INFO:lda:<30> log likelihood: -343791
INFO:lda:<40> log likelihood: -341480
INFO:lda:<50> log likelihood: -340465
INFO:lda:<60> log likelihood: -339301
INFO:lda:<70> log likelihood: -338689
INFO:lda:<80> log likelihood: -338323
INFO:lda:<90> log likelihood: -337856
INFO:lda:<100> log likelihood: -338053
INFO:lda:<110> log likelihood: -338210
INFO:lda:<120> log likelihood: -337836
INFO:lda:<130> log likelihood: -337579
INFO:lda:<140> log likelihood: -337681
INFO:lda:<150> log likelihood: -337366
INFO:lda:<160> log likelihood: -337547
INFO:lda:<170> log likelihood: -337427
INFO:lda:<180> log likelihood: -337108
INFO:lda:<190> log likelihood: -337348
INFO:lda:<200> log likelihood: -337119
INFO:lda:<210> log likelihood: -336676
INFO:lda:<220> log likelihood: -3366

shape: (400, 5)
stats of topics amony documents:
{2: 111, 4: 67, 0: 45, 3: 89, 1: 88}


In [2]:
    for i, topic_dist in enumerate(topic_word):
        topic_words = np.array(featureNames)[np.argsort(topic_dist)][:-(numTopic+5):-1]
        print('*Topic {}\n- {}'.format(i, ','.join(topic_words)))

*Topic 0
- BS,SS,IEEE,CX,HARQ,REQ,IP,ACK,RSP
*Topic 1
- DL,IEEE,UL,IE,MAP,AAS,BS,SS,OFDMA
*Topic 2
- IEEE,MAC,QPSK,QAM,PHY,OFDMA,SS,MBS,MHz
*Topic 3
- RS,MS,BS,MR,IEEE,CID,REQ,RSP,RSs
*Topic 4
- BS,MOB,MSS,IEEE,MS,REQ,HO,RSP,ID


In [3]:
#extracting file Names in range of 400-1400 
from fileProcessing import *
pdfDir =  "F:\\WireLessNLPGRA\\80216CompProj"#change this to the pdf folders
fileName = getSelectedFileNames(pdfDir, 400, 1400)
pickle.dump(fileName, open("1000Files.dat", "wb"))

loading previous  randomIdx.dat  success


In [2]:
fileName

['F:\\WireLessNLPGRA\\80216CompProj\\C80216j-07_451.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216e-04_132.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216g-05_028.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216e-04_01.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216e-04_330.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216d-04_101.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216e-04_209r2.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216g-06_040r1.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216e-05_303r1.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216j-07_264r3.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216h-07_027r2.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\802161c-00_15.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216j-06_182.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216j-07_073r3.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216j-07_034.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216maint-04_09.pdf',
 'F:\\WireLessNLPGRA\\80216CompProj\\C80216j-06_044.pdf',
 